# Structured Output Example

## Installations

In [ ]:
# !pip install --quiet --force-reinstall prompttools

## Setup imports and API keys

First, we'll need to set our API keys. If we are in DEBUG mode, we don't need to use real OpenAI or Hegel AI API keys, so for now we'll set them to empty strings.

In [ ]:
import os
os.environ['DEBUG'] = "1"  # Set to "1" if you want to use debug mode.
os.environ['HEGELAI_API_KEY'] = ""  # Optional, it will be needed to use with `HegelScribe` to persist/visualize your experiments
os.environ['OPENAI_API_KEY'] = ""

Then we'll import the relevant `prompttools` modules to setup our experiment.

In [ ]:
from typing import Dict, Tuple
from prompttools.harness import PromptTemplateExperimentationHarness
from prompttools.experiment import OpenAICompletionExperiment

## Run experiments

Next, we create our test inputs. For this example, we'll use a prompt template, which uses [jinja](https://jinja.palletsprojects.com/en/3.1.x/) for templating.

In [ ]:
prompt_templates = ["Generate valid JSON from the following input: {{input}}", "Generate valid python to complete the following task: {{input}}"]
user_inputs = [{"input": "The task is to count all the words in a string"}, {"input": "The task is to add up numbers 1 to 100"}]

Now we can define an experimentation harness for our inputs and model. We could also pass model arguments if, for example, we wanted to change the model temperature.

In [ ]:
harness = PromptTemplateExperimentationHarness(OpenAICompletionExperiment,
                                               "text-davinci-003", 
                                               prompt_templates, 
                                               user_inputs,
                                               # Zero temperature is better for
                                               # structured outputs
                                               model_arguments={"temperature": 0})

We can then run the experiment to get results.

In [ ]:
harness.run()
harness.visualize()

You can use the `pivot` keyword argument to view results by the template and inputs that created them.

In [ ]:
harness.visualize(pivot=True)

## Evaluate the model response

To evaluate the results, we'll define an eval function. We can use the json and python utilities to validate our responses.

In [ ]:
from prompttools.utils import json
from prompttools.utils import python

def extract_responses(output) -> str:
    rsps = [choice["text"].strip() for choice in output["choices"]]
    return rsps


def is_json(
    input_pair: Tuple[str, Dict[str, str]], results: Dict, metadata: Dict
) -> float:
    """
    A simple test that checks semantic similarity between the user input
    and the model's text responses.
    """
    validations = [json.validate(response) for response in extract_responses(results)]
    return min(validations)

def is_python(
    input_pair: Tuple[str, Dict[str, str]], results: Dict, metadata: Dict
) -> float:
    """
    A simple test that checks semantic similarity between the user input
    and the model's text responses.
    """
    validations = [python.validate(response) for response in extract_responses(results)]
    return min(validations)


Finally, we can evaluate and visualize the results.

In [ ]:
harness.evaluate("is_json", is_json)
harness.evaluate("is_python", is_python)
harness.visualize()